<a href="https://colab.research.google.com/github/rrizwan98/SSUET-chatbot/blob/main/chatbot_LSTM_Tflearn.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install nltk
!pip install tflearn

     |████████████████████████████████| 107 kB 9.3 MB/s 
  Created wheel for tflearn: filename=tflearn-0.5.0-py3-none-any.whl size=127299 sha256=5e5339bf603ca90fb989c534c74400bfa37de8591ee27631a4c12f6cfe717d41
  Stored in directory: /root/.cache/pip/wheels/5f/14/2e/1d8e28cc47a5a931a2fb82438c9e37ef9246cc6a3774520271
Successfully built tflearn


In [2]:
# from google.colab import drive
# drive.mount('/content/drive')

In [99]:
import nltk
from nltk.stem.lancaster import LancasterStemmer
stemmer = LancasterStemmer()

import numpy
import tflearn
import tensorflow
import random

import json
with open('/content/intents3.json') as file:
    data = json.load(file)


In [100]:
print(data)

{'intents': [{'context': 'greeting', 'question': ['Hi', 'How are you', 'Is anyone there?', 'Hello', 'Good day'], 'answer': ['Hello!', 'Good to see you again!', 'Hi there, how can I help?', 'Assalam-o-alaikum', 'I am SSSUET AI chat Solve the student qurees'], 'context_set': ''}, {'context': 'goodbye', 'question': ['cya', 'See you later', 'Goodbye', 'I am Leaving', 'Have a Good day'], 'answer': ['Sad to see you go:', 'Talk to you later', 'Goodbye!', 'bye bye', 'Allah Hafiz'], 'context_set': ''}, {'context': 'Introduction to computing itc techer name', 'question': ['Introduction to computing teacher name?'], 'answer': ['itc teacher name: Sir Haris Mehboob'], 'context_set': ''}, {'context': 'Introduction to computing/ itc credit hours', 'question': ['Introduction to computing CREDIT HOURS?'], 'answer': [' ITC CREDIT HOURS:3+1.'], 'context_set': ''}, {'context': 'Introduction to computing itc techer name', 'question': ['itc teacher room?'], 'answer': ['itc teacher room No:___undefine____'],

In [101]:
print(data['intents'])

[{'context': 'greeting', 'question': ['Hi', 'How are you', 'Is anyone there?', 'Hello', 'Good day'], 'answer': ['Hello!', 'Good to see you again!', 'Hi there, how can I help?', 'Assalam-o-alaikum', 'I am SSSUET AI chat Solve the student qurees'], 'context_set': ''}, {'context': 'goodbye', 'question': ['cya', 'See you later', 'Goodbye', 'I am Leaving', 'Have a Good day'], 'answer': ['Sad to see you go:', 'Talk to you later', 'Goodbye!', 'bye bye', 'Allah Hafiz'], 'context_set': ''}, {'context': 'Introduction to computing itc techer name', 'question': ['Introduction to computing teacher name?'], 'answer': ['itc teacher name: Sir Haris Mehboob'], 'context_set': ''}, {'context': 'Introduction to computing/ itc credit hours', 'question': ['Introduction to computing CREDIT HOURS?'], 'answer': [' ITC CREDIT HOURS:3+1.'], 'context_set': ''}, {'context': 'Introduction to computing itc techer name', 'question': ['itc teacher room?'], 'answer': ['itc teacher room No:___undefine____'], 'context_se

In [102]:
nltk.download('punkt')

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


True

In [103]:
words = []
labels = []
docs_x = []
docs_y = []

# Tokinization.
for intent in data['intents']:
    for pattern in intent['question']:
      wrds = nltk.word_tokenize(pattern)
      words.extend(wrds)
      docs_x.append(wrds)
      docs_y.append(intent["context"])
        
    if intent['context'] not in labels:
        labels.append(intent['context'])

In [104]:
# docs_x

In [105]:
# docs_y

In [106]:
words = [stemmer.stem(w.lower()) for w in words if w != "?"]
words = sorted(list(set(words)))

labels = sorted(labels)

In [107]:
# labels

In [108]:
# beg of words.
training = []
output = []

out_empty = [0 for _ in range(len(labels))]

for x, doc in enumerate(docs_x):
    bag = []

    wrds = [stemmer.stem(w.lower()) for w in doc]

    for w in words:

      if w in wrds:
          bag.append(1)
      else:
          bag.append(0)

    output_row = out_empty[:]
    output_row[labels.index(docs_y[x])] = 1

    training.append(bag)
    output.append(output_row)

In [109]:
# print(len(training))
# print(len(output))

In [112]:
# from tflearn.data_utils import to_categorical, pad_sequences
# trainX = pad_sequences(training, maxlen=100, value=0.)
# testX = pad_sequences(output, maxlen=100, value=0.)

In [113]:
# trainY = to_categorical(trainY)
# testY = to_categorical(testY)

In [114]:
training = numpy.array(training)
output = numpy.array(output)
print(len(training[0]))
print(len(output[0]))

111
210


In [128]:
# Network building
from tensorflow.python.framework import ops
ops.reset_default_graph()
net = tflearn.input_data([None, len(training[0])])
net = tflearn.embedding(net, input_dim=10000, output_dim=128)
net = tflearn.lstm(net, 64,return_seq=True)
# net= tflearn.dropout(net, 0.2)
net = tflearn.lstm(net, 128)
net = tflearn.fully_connected(net,len(output[0]) , activation='softmax')
net = tflearn.regression(net, optimizer='adam', learning_rate=0.001,loss='categorical_crossentropy')
model = tflearn.DNN(net)

In [124]:
# from tensorflow.python.framework import ops
# ops.reset_default_graph()
# net = tflearn.input_data(shape=[None, len(training[0])])
# net = tflearn.fully_connected(net, 8)
# net = tflearn.fully_connected(net, 8)
# net = tflearn.fully_connected(net, len(output[0]), activation="softmax")
# net = tflearn.regression(net,optimizer='adam', learning_rate=0.001,loss='categorical_crossentropy')

# model = tflearn.DNN(net)


In [129]:
# model.fit(training, output, n_epoch=250, batch_size=8, show_metric=True)
model.fit(training, output, show_metric=True,n_epoch=250,batch_size=32,validation_set=0.1)

Training Step: 1749  | total loss: 0.20949 | time: 0.603s
| Adam | epoch: 250 | loss: 0.20949 - acc: 0.9679 -- iter: 192/213
Training Step: 1750  | total loss: 0.20345 | time: 1.713s
| Adam | epoch: 250 | loss: 0.20345 - acc: 0.9648 | val_loss: 12.77206 - val_acc: 0.0417 -- iter: 213/213
--


In [130]:
# model.save("chatbot.tflearn")

In [131]:
def bag_of_words(s, words):
    bag = [0 for _ in range(len(words))]

    s_words = nltk.word_tokenize(s)
    s_words = [stemmer.stem(word.lower()) for word in s_words]

    for se in s_words:
        for i, w in enumerate(words):
            if w == se:
                bag[i] = 1
            
    return numpy.array(bag)


def chat():
    print("Start talking with the bot (type quit to stop)!")
    while True:
        inp = input("You: ")
        if inp.lower() == "quit":
            break

        results = model.predict([bag_of_words(inp, words)])
        results_index = numpy.argmax(results)
        tag = labels[results_index]

        for tg in data["intents"]:
            if tg['context'] == tag:
                responses = tg['answer']

        print(responses)

chat()

Start talking with the bot (type quit to stop)!
You: ap teacher room
['Basic Electronics teacher room:__undefine__']
You: Basic Electronics teacher room
['Basic Electronics teacher room:__undefine__']
You: AP teacher room
['Basic Electronics teacher room:__undefine__']
You: what is DSA lab teacher name
['Programming Fundamental lab teacher Name: Miss Nudrat Naveed .']
You: DSA Lab room no
['Computer Organization & Architecture class room:CG-06']
You: quit
